<a href="https://colab.research.google.com/github/Ayush310803/ViT/blob/main/Deeplabv3%2B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import zipfile

os.environ["KAGGLE_CONFIG_DIR"] = "/content"
!kaggle datasets download -d rajkumarl/people-clothing-segmentation

dataset_zip = "/content/people-clothing-segmentation.zip"
extract_path = "/content/dataset"
os.makedirs(extract_path, exist_ok=True)

with zipfile.ZipFile(dataset_zip, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("✅ Dataset extracted successfully.")
!ls /content/dataset

image_dir = "/content/dataset/jpeg_images/IMAGES"
mask_dir = "/content/dataset/jpeg_masks/MASKS"
label_path = "/content/dataset/labels.csv"

Dataset URL: https://www.kaggle.com/datasets/rajkumarl/people-clothing-segmentation
License(s): CC0-1.0
people-clothing-segmentation.zip: Skipping, found more recently modified local copy (use --force to force download)
✅ Dataset extracted successfully.
 jpeg_images   jpeg_masks  'labels (1).csv'   labels.csv   png_images   png_masks


In [2]:
!pip install tensorflow keras opencv-python matplotlib

In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

labels_df = pd.read_csv(label_path)

image_paths = [os.path.join(image_dir, img) for img in os.listdir(image_dir)]
mask_paths = [os.path.join(mask_dir, mask) for mask in os.listdir(mask_dir)]

image_paths.sort()
mask_paths.sort()

images = np.array([cv2.resize(cv2.imread(img_path, cv2.IMREAD_COLOR), (512, 512)) for img_path in image_paths])
masks = np.array([cv2.resize(cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE), (512, 512)) for mask_path in mask_paths])

images = images / 255.0
num_classes = len(np.unique(masks))
masks = to_categorical(masks, num_classes=num_classes)
X_train, X_test, y_train, y_test = train_test_split(images, masks, test_size=0.2, random_state=42)

In [ ]:
from tensorflow.keras import layers, Model
from tensorflow.keras.applications import ResNet50

def build_deeplabv3_plus(input_shape, num_classes):
    inputs = layers.Input(shape=input_shape)

    resnet50 = ResNet50(weights='imagenet', include_top=False, input_tensor=inputs)
    x = resnet50.get_layer('conv4_block6_out').output

    b1 = layers.Conv2D(256, 1, padding='same', use_bias=False)(x)
    b1 = layers.BatchNormalization()(b1)
    b1 = layers.Activation('relu')(b1)

    b2 = layers.Conv2D(256, 3, padding='same', dilation_rate=6, use_bias=False)(x)
    b2 = layers.BatchNormalization()(b2)
    b2 = layers.Activation('relu')(b2)

    b3 = layers.Conv2D(256, 3, padding='same', dilation_rate=12, use_bias=False)(x)
    b3 = layers.BatchNormalization()(b3)
    b3 = layers.Activation('relu')(b3)

    b4 = layers.Conv2D(256, 3, padding='same', dilation_rate=18, use_bias=False)(x)
    b4 = layers.BatchNormalization()(b4)
    b4 = layers.Activation('relu')(b4)

    x = layers.concatenate([b1, b2, b3, b4])
    x = layers.Conv2D(256, 1, padding='same', use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.UpSampling2D(size=(4, 4), interpolation='bilinear')(x)
    outputs = layers.Conv2D(num_classes, 1, activation='softmax')(x)
    model = Model(inputs, outputs)
    return model

input_shape = (512, 512, 3)
model = build_deeplabv3_plus(input_shape, num_classes)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(X_train, y_train, batch_size=8, epochs=50, validation_data=(X_test, y_test))

In [ ]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

In [ ]:
model.save('deeplabv3_plus_model.h5')
from tensorflow.keras.models import load_model
loaded_model = load_model('deeplabv3_plus_model.h5')